In [1]:
!pip install -U tokenizers
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 3.2 MB 1.2 MB/s eta 0:00:01


In [ ]:
vocab_length = 50000


In [2]:
!wget --header="Host: codeload.github.com" \
    --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36" \
    --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" \
    --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" \
    --header="Referer: https://github.com/google-research/bert" \
    --header="Cookie: _octo=GH1.1.68793831.1588906101; _ga=GA1.2.19990328.1588906163; logged_in=no; _gat=1; tz=Asia%2FKarachi" \
    --header="Connection: keep-alive" "https://codeload.github.com/google-research/bert/zip/master" \
    -c -O 'bert-master.zip'

!unzip bert-master.zip
!rm bert-master.zip

--2021-02-13 21:30:59--  https://codeload.github.com/google-research/bert/zip/master
Resolving codeload.github.com (codeload.github.com)... 13.233.43.20
Connecting to codeload.github.com (codeload.github.com)|13.233.43.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘bert-master.zip’

bert-master.zip         [  <=>               ] 106.44K   489KB/s    in 0.2s    

2021-02-13 21:31:00 (489 KB/s) - ‘bert-master.zip’ saved [108997]

Archive:  bert-master.zip
eedf5716ce1268e56f0a50264a88cafad334ac61
   creating: bert-master/
  inflating: bert-master/.gitignore  
  inflating: bert-master/CONTRIBUTING.md  
  inflating: bert-master/LICENSE     
  inflating: bert-master/README.md   
  inflating: bert-master/__init__.py  
  inflating: bert-master/create_pretraining_data.py  
  inflating: bert-master/extract_features.py  
  inflating: bert-master/modeling.py  
  inflating: bert-master/modeling_test.py  
  inflating: bert-maste

In [3]:
!wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip \
    -c -O 'bert-base-uncased.zip'

!unzip bert-base-uncased.zip -d bert-base-uncased
!rm bert-base-uncased.zip


--2021-02-13 21:31:50--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.21.48, 216.58.207.16, 216.58.207.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.21.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 408102251 (389M) [application/zip]
Saving to: ‘bert-base-uncased.zip’

bert-base-uncased.z 100%[===================>] 389.20M  1.73MB/s    in 3m 2s   

2021-02-13 21:34:53 (2.14 MB/s) - ‘bert-base-uncased.zip’ saved [408102251/408102251]

Archive:  bert-base-uncased.zip
  inflating: bert-base-uncased/bert_model.ckpt.data-00000-of-00001  
  inflating: bert-base-uncased/bert_config.json  
  inflating: bert-base-uncased/vocab.txt  
  inflating: bert-base-uncased/bert_model.ckpt.index  


In [ ]:
import glob

files = glob.glob('data/' + '*')

text_data = []
for file in files:
    with open(file, 'r') as data:
        text = list(filter(lambda x: x != '\n', data.readlines()))
        text_data.append(''.join(text))

with open('all_data.txt','w') as f:
    f.write('\n'.join(text_data))

In [4]:
import tokenizers

roman_BWPT = tokenizers.BertWordPieceTokenizer(
    # add_special_tokens=True, # This argument doesn't work in the latest version of BertWordPieceTokenizer
    unk_token='[UNK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True,
    lowercase=True,
    wordpieces_prefix='##'
)

roman_BWPT.train(
    files=["all_data.txt"],
    vocab_size=vocab_length,
    min_frequency=3,
    limit_alphabet=1000,
    show_progress=True,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]']
)

roman_BWPT.save_model(".", "roman-urdu-vocab-uncased_"+str(vocab_length))

['./roman-urdu-vocab-uncased_20K-vocab.txt']

In [ ]:
english_vocab = open('bert-base-uncased/vocab.txt', 'r').read().split('\n')
roman_urdu_vocab = open('roman-urdu-vocab-uncased_'+str(vocab_length)+'-vocab.txt', 'r').read().split('\n')

common_vocab = list(set(english_vocab).intersection(set(roman_urdu_vocab)))
print('No. of common tokens: ',len(common_vocab))

augmented_vocab = [''] * len(roman_urdu_vocab)

for vocab in common_vocab:
    augmented_vocab[english_vocab.index(vocab)] = vocab
    roman_urdu_vocab.pop(roman_urdu_vocab.index(vocab))

for i in range(len(augmented_vocab)):
    if augmented_vocab[i] == '':
        augmented_vocab[i] = roman_urdu_vocab.pop(0)

with open('augmented_vocab.txt', 'w') as v:
    v.write('\n'.join(augmented_vocab))


In [5]:
BERT_BASE_DIR='bert-base-uncased'

import json
with open(BERT_BASE_DIR+'/bert_config.json', "r+") as jsonFile:
    data = json.load(jsonFile)
    data["vocab_size"] = sum(1 for line in open('augmented_vocab.txt'))
    jsonFile.seek(0)  # rewind
    json.dump(data, jsonFile)
    jsonFile.truncate()

In [ ]:
!python bert-master/create_pretraining_data.py \
    --input_file all_data.txt \
    --output_file tf_examples_multi.tfrecord \
    --vocab_file augmented_vocab.txt \
    --do_lower_case True \
    --max_seq_length 128 \
    --max_predictions_per_seq 20 \
    --masked_lm_prob 0.15 \
    --random_seed 42 \
    --dupe_factor 5

In [10]:
!python bert-master/run_pretraining.py \
    --input_file=tf_examples_multi.tfrecord \
    --output_dir=bert_bilingual_roman_urdu \
    --do_train=True \
    --do_eval=True \
    --bert_config_file=$BERT_BASE_DIR/bert_config.json \
    --train_batch_size=32 \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --num_train_steps=100000 \
    --num_warmup_steps=10 \
    --learning_rate=2e-5 \
    --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \




W1115 20:56:44.335807 140718067062592 module_wrapper.py:139] From bert-master/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1115 20:56:44.336080 140718067062592 module_wrapper.py:139] From bert-master/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1115 20:56:44.336295 140718067062592 module_wrapper.py:139] From /mnt/3E56E27B56E232F7/Users/usama/PycharmProjects/AIM_LAB/Roman_Urdu_BERT/bert-master/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Traceback (most recent call last):
  File "bert-master/run_pretraining.py", line 493, in <module>
    tf.app.run()
  File "/home/usamathescientist/Desktop/PycharmProjects/AIM_LAB/Roman_Urdu_BERT/venv/lib/python3.6/site-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_un